In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [81]:
def fetch_logged_data(run_id):
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

In [74]:
features = ["acousticness", "danceability", "energy", "explicit"]

In [82]:
mlflow.sklearn.autolog()

In [83]:
# Scale train and test data
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [84]:
forest isnullndomForestClassifier(n_estimators=10)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

In [85]:
with mlflow.start_run() as run:
    multi_target_forest.fit(X_train, y_train)

2021/03/11 19:04:39 WARNING mlflow.sklearn.utils: log_loss failed. The metric training_log_losswill not be recorded. Metric error: Found array with dim 3. Estimator expected <= 2.
2021/03/11 19:04:39 WARNING mlflow.sklearn.utils: roc_auc_score failed. The metric training_roc_auc_scorewill not be recorded. Metric error: Found array with dim 3. Estimator expected <= 2.
2021/03/11 19:04:39 WARNING mlflow.sklearn.utils: plot_confusion_matrix failed. The artifact training_confusion_matrix will not be recorded. Artifact error: multilabel-indicator is not supported
2021/03/11 19:04:40 WARNING mlflow.utils.autologging_utils: MLflow issued a warning during sklearn autologging: "/usr/local/lib/python3.7/site-packages/mlflow/models/signature.py:123: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement er

In [93]:
params, metrics, tags, artifacts = fetch_logged_data(run.info.run_id)

In [94]:
scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']

In [95]:
target_names = df_target.columns.tolist()

In [96]:
y_pred = multi_target_forest.predict(X_test)
report = classification_report(y_test, y_pred, target_names=target_names)

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [97]:
print(report)

                          precision    recall  f1-score   support

        Francisco Canaro       0.93      0.60      0.73       439
Tadeusz Dolega Mostowicz       1.00      0.95      0.97       290
        Эрнест Хемингуэй       1.00      0.98      0.99       249
       Эрих Мария Ремарк       1.00      0.98      0.99       199
         Frédéric Chopin       0.88      0.42      0.57       198
   Johann Sebastian Bach       0.98      0.28      0.44       182
 Wolfgang Amadeus Mozart       0.85      0.06      0.11       185
    Ludwig van Beethoven       0.76      0.09      0.17       171
           Frank Sinatra       0.54      0.04      0.08       167
   Wiener Philharmoniker       0.96      0.37      0.54       139

               micro avg       0.96      0.54      0.69      2219
               macro avg       0.89      0.48      0.56      2219
            weighted avg       0.90      0.54      0.62      2219
             samples avg       0.03      0.03      0.03      2219



In [121]:
X_train

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
120506,0.86900,0.465,212000,0.2770,0,0.000000,2,0.157,-13.384,1,45,0.0306,105.451,0.664,1990
44830,0.89900,0.425,378760,0.2990,0,0.700000,9,0.736,-13.243,1,0,0.0574,127.038,0.298,1952
168704,0.92500,0.398,194867,0.0977,0,0.000003,1,0.160,-18.775,1,31,0.0391,135.056,0.172,1993
69701,0.40300,0.787,178453,0.6220,0,0.000004,0,0.219,-11.738,1,41,0.0532,86.438,0.932,1986
15524,0.43400,0.709,216693,0.7570,0,0.000000,7,0.336,-7.006,1,60,0.0408,140.664,0.852,1998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73349,0.26200,0.624,293227,0.7070,0,0.000000,0,0.963,-8.233,1,65,0.0291,121.961,0.368,2004
109259,0.00153,0.628,227068,0.9370,0,0.757000,1,0.107,-4.491,1,0,0.0506,147.006,0.848,2014
50057,0.24200,0.332,159453,0.7240,0,0.000000,2,0.614,-8.829,1,57,0.0336,159.715,0.938,1979
5192,0.91700,0.334,731293,0.4190,0,0.000702,8,0.235,-11.443,1,10,0.0348,108.368,0.468,1946


In [117]:
X_train["mode"].unique()

array([1, 0])

In [98]:
clf = mlflow.sklearn.load_model("mlruns/0/0f10aa9cdc2c476eab4796bbf5a50b62/artifacts/model/")

In [104]:
clf

MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10),
                      n_jobs=-1)

In [106]:
clf = mlflow.pyfunc.load_model("mlruns/0/0f10aa9cdc2c476eab4796bbf5a50b62/artifacts/model/")

In [165]:
data_dict = {'acousticness': 0.909, 'danceability': 0.212, 'duration_ms': 70453, 'energy': 0.0473, 'explicit': 0, 'instrumentalness': 0.927, 'key': 5, 'liveness': 0.0958, 'loudness': -27.82800000000001, 'mode': 1, 'popularity': 0, 'speechiness': 0.04, 'tempo': 110.388, 'valence': 0.248, 'year': 1952}

In [166]:
data = pd.DataFrame(data_dict, index=[0])

In [177]:
preds = clf.predict(X_test)

In [194]:
preds[0][0] = 1

In [196]:
preds[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [199]:
y_test.columns

Index(['Francisco Canaro', 'Tadeusz Dolega Mostowicz', 'Эрнест Хемингуэй',
       'Эрих Мария Ремарк', 'Frédéric Chopin', 'Johann Sebastian Bach',
       'Wolfgang Amadeus Mozart', 'Ludwig van Beethoven', 'Frank Sinatra',
       'Wiener Philharmoniker'],
      dtype='object')

In [ ]:
X_test